## Xavier and He Initialization

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [4]:
# He initialization with a uniform distribution

he_avg_init = keras.initializers.VarianceScaling(scale = 2., mode = "fan_avg",
                                                distribution = "uniform")
keras.layers.Dense(10, activation = "sigmoid", kernel_initializer = he_avg_init)

In [10]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

# leaky ReLU

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [9]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7902
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8066
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5832 - accuracy: 0.8075 - val_loss: 0.5582 - val_accuracy: 0.8200
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5553 - accuracy: 0.8156 - val_loss: 0.5349 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5338 - accuracy: 0.8224 - val_loss: 0.5156 - val_accuracy:

In [10]:
# PReLU

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [12]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.3461 - accuracy: 0.6209 - val_loss: 0.9255 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8197 - accuracy: 0.7355 - val_loss: 0.7305 - val_accuracy: 0.7628
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6965 - accuracy: 0.7693 - val_loss: 0.6565 - val_accuracy: 0.7880
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6331 - accuracy: 0.7909 - val_loss: 0.6003 - val_accuracy: 0.8048
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5917 - accuracy: 0.8057 - val_loss: 0.5656 - val_accuracy: 0.8180
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5618 - accuracy: 0.8136 - val_loss: 0.5406 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5390 - accuracy: 0.8205 - val_loss: 0.5196 - val_accuracy:

In [16]:
from scipy.special import erfc

# alpha and scale to self normalize with mean 0 and standard deviation 1
# (see equation 14 in the paper):
alpha_0_1 = -np.sqrt(2 / np.pi) / (erfc(1/np.sqrt(2)) * np.exp(1/2) - 1)
scale_0_1 = (1 - erfc(1 / np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2 * np.pi) * (2 * erfc(np.sqrt(2))*np.e**2 + np.pi*erfc(1/np.sqrt(2))**2*np.e - 2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e)+np.pi+2)**(-1/2)

In [17]:
def selu(z, scale=scale_0_1, alpha=alpha_0_1):
    return scale * elu(z, alpha)

In [18]:
keras.layers.Dense(10, activation="selu",
                   kernel_initializer="lecun_normal")

In [19]:
np.random.seed(42)
tf.random.set_seed(42)

In [20]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [21]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [11]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [23]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 18s 9ms/step - loss: 1.1344 - accuracy: 0.5573 - val_loss: 1.1278 - val_accuracy: 0.4936
Epoch 2/5
1719/1719 [==============================] - 16s 9ms/step - loss: 0.8060 - accuracy: 0.6918 - val_loss: 0.7098 - val_accuracy: 0.7254
Epoch 3/5
1719/1719 [==============================] - 16s 9ms/step - loss: 0.6977 - accuracy: 0.7395 - val_loss: 0.6509 - val_accuracy: 0.7710
Epoch 4/5
1719/1719 [==============================] - 16s 9ms/step - loss: 0.6434 - accuracy: 0.7605 - val_loss: 0.5453 - val_accuracy: 0.7982
Epoch 5/5
1719/1719 [==============================] - 16s 9ms/step - loss: 0.5673 - accuracy: 0.7885 - val_loss: 0.5220 - val_accuracy: 0.8074


## Batch Normalization

In [24]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.BatchNormalization(), # batch normalization before each hidden layer's activation function
    keras.layers.Dense(300, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation = "softmax")
])

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_110 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_111 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [26]:
[(var.name, var.trainable) for var in model.layers[1].variables]
# first BN layer
# 2 are trainable while 2 are not

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [28]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False), # remove bias term from previous layer
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

## Gradient Clipping

In [29]:
optimizer = keras.optimizers.SGD(clipvalue = 1.0)
model.compile(loss = "mse", optimizer = optimizer)

## Transfering Learning with Keras

In [30]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [31]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [32]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [33]:
tf.random.set_seed(42)
np.random.seed(42)

In [34]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [35]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [36]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 4s 2ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3892 - val_accuracy: 0.8675
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3523 - accuracy: 0.8788 - val_loss: 0.3290 - val_accuracy: 0.8819
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3170 - accuracy: 0.8896 - val_loss: 0.3012 - val_accuracy: 0.8989
Epoch 4/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2973 - accuracy: 0.8973 - val_loss: 0.2895 - val_accuracy: 0.9023
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2835 - accuracy: 0.9020 - val_loss: 0.2774 - val_accuracy: 0.9066
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2729 - accuracy: 0.9060 - val_loss: 0.2732 - val_accuracy: 0.9066
Epoch 7/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2641 - accuracy: 0.9090 - val_loss: 0.2721 - val_accuracy:

In [37]:
model_A.save("my_model_A.h5")

In [38]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [39]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [40]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 38ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 15ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 16ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 16ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 16ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 16ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 19ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [41]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # reuse all layers except for the output layer
model_B_on_A.add(keras.layers.Dense(1, activation = "sigmoid"))

In [42]:
# when training model_b_on_A, it affects model_A
# to avoid, use clone model_A before reusing its layers

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # reuse all layers except for the output layer
model_B_on_A.add(keras.layers.Dense(1, activation = "sigmoid"))

In [45]:
# freese the reused layers during the first few epochs, giving the new layer some time to learn reasonable weights

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss = "binary_crossentropy", optimizer = "sgd",
                    metrics = ["accuracy"])

# train the model for few epochs, then unfreeze the reused layers

history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 4,
                          validation_data = (X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
    
optimizer = keras.optimizers.SGD(learning_rate = 1e-4)
model_B_on_A.compile(loss = "binary_crossentropy", optimizer = optimizer,
                    metrics = ["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 16,
                          validation_data = (X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 33ms/step - loss: 0.0810 - accuracy: 0.9900 - val_loss: 0.0969 - val_accuracy: 0.9878
Epoch 2/4
7/7 [==============================] - 0s 13ms/step - loss: 0.0724 - accuracy: 0.9950 - val_loss: 0.0894 - val_accuracy: 0.9868
Epoch 3/4
7/7 [==============================] - 0s 13ms/step - loss: 0.0655 - accuracy: 0.9950 - val_loss: 0.0832 - val_accuracy: 0.9868
Epoch 4/4
7/7 [==============================] - 0s 13ms/step - loss: 0.0598 - accuracy: 0.9950 - val_loss: 0.0782 - val_accuracy: 0.9878
Epoch 1/16
7/7 [==============================] - 1s 36ms/step - loss: 0.0566 - accuracy: 0.9950 - val_loss: 0.0778 - val_accuracy: 0.9878
Epoch 2/16
7/7 [==============================] - 0s 16ms/step - loss: 0.0562 - accuracy: 0.9950 - val_loss: 0.0774 - val_accuracy: 0.9878
Epoch 3/16
7/7 [==============================] - 0s 17ms/step - loss: 0.0557 - accuracy: 0.9950 - val_loss: 0.0770 - val_accuracy: 0.9878
Epoch 4/16
7/7 [===============

In [47]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0627 - accuracy: 0.9935


[0.06274233013391495, 0.9934999942779541]

In [48]:
# Momentum Optimization

optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9)

In [49]:
# Nesterov Accelerated Gradient

optimizer = keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9, nesterov = True)

In [50]:
# AdaGrad

optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

In [51]:
# RMSProp

optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [52]:
# Adam Optimization

optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [53]:
# Adamax Optimization

optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [54]:
# Nadam Optimization

optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## Avoiding Overfitting Through Regularization

In [4]:
# l1 and l2 Regularization

# l2
layer = keras.layers.Dense(100, activation = "elu",
                          kernel_initializer = "he_normal",
                          kernel_regularizer = keras.regularizers.l2(0.01))

# keras.regularizers.l1() for l1 regularization
# keras.regularizers.l1_l2() for both

In [8]:
# apply same regularizer to all layers in network

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2

In [9]:
# dropout rate

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(300, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(100, activation = "elu", kernel_initializer = "he_normal"),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(10, activation = "softmax")
])

In [13]:
# Monte Carlo (MC) Dropout = averaging over multiple prediction with dropout

import numpy as np
y_probas = np.stack([model(X_test_scaled, training = True)
                    for sample in range(100)])
y_proba = y_probas.mean(axis = 0)

In [14]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0.13, 0.08, 0.05, 0.1 , 0.12, 0.07, 0.21, 0.03, 0.11, 0.1 ]],
      dtype=float32)

In [15]:
np.round(y_probas[:, :1], 2)

array([[[0.04, 0.1 , 0.09, 0.01, 0.06, 0.01, 0.49, 0.  , 0.18, 0.01]],

       [[0.06, 0.05, 0.06, 0.01, 0.02, 0.04, 0.1 , 0.47, 0.16, 0.02]],

       [[0.03, 0.09, 0.05, 0.31, 0.06, 0.02, 0.17, 0.02, 0.22, 0.04]],

       [[0.04, 0.35, 0.13, 0.17, 0.02, 0.02, 0.06, 0.  , 0.19, 0.02]],

       [[0.14, 0.06, 0.02, 0.06, 0.05, 0.03, 0.19, 0.01, 0.36, 0.07]],

       [[0.11, 0.01, 0.06, 0.02, 0.03, 0.08, 0.38, 0.  , 0.16, 0.15]],

       [[0.05, 0.02, 0.05, 0.02, 0.32, 0.04, 0.32, 0.  , 0.02, 0.17]],

       [[0.16, 0.21, 0.02, 0.07, 0.03, 0.03, 0.2 , 0.06, 0.09, 0.12]],

       [[0.06, 0.13, 0.01, 0.04, 0.33, 0.08, 0.05, 0.14, 0.15, 0.01]],

       [[0.  , 0.01, 0.  , 0.02, 0.02, 0.  , 0.14, 0.  , 0.08, 0.73]],

       [[0.14, 0.06, 0.  , 0.03, 0.17, 0.18, 0.17, 0.02, 0.22, 0.01]],

       [[0.07, 0.1 , 0.01, 0.1 , 0.01, 0.06, 0.32, 0.03, 0.3 , 0.01]],

       [[0.09, 0.19, 0.06, 0.18, 0.03, 0.08, 0.01, 0.01, 0.19, 0.15]],

       [[0.05, 0.  , 0.08, 0.05, 0.18, 0.25, 0.18, 0.01, 0.05, 0

In [16]:
np.round(y_proba[:1], 2)

array([[0.11, 0.1 , 0.06, 0.09, 0.11, 0.07, 0.2 , 0.05, 0.11, 0.11]],
      dtype=float32)

In [17]:
# standard deviation of the probability estimates

y_std = y_probas.std(axis = 0)
np.round(y_std[:1], 2)

array([[0.16, 0.13, 0.07, 0.11, 0.13, 0.08, 0.18, 0.1 , 0.11, 0.12]],
      dtype=float32)

In [19]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.1167

In [20]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [21]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [22]:
tf.random.set_seed(42)
np.random.seed(42)

In [23]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [24]:
mc_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dropout (Dropout)           (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 300)               235500    
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_9 (Dense)             (None, 100)               30100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 10)               

In [25]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [26]:
mc_model.set_weights(model.get_weights())

In [27]:
np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

array([[0.13, 0.08, 0.05, 0.1 , 0.12, 0.07, 0.21, 0.03, 0.11, 0.1 ]],
      dtype=float32)

In [29]:
# Max-Norm Regularization

layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

In [30]:
MaxNormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(300),
    MaxNormDense(100),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4738 - accuracy: 0.8335 - val_loss: 0.3767 - val_accuracy: 0.8622
Epoch 2/2
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3538 - accuracy: 0.8718 - val_loss: 0.3763 - val_accuracy: 0.8694
